<a href="https://colab.research.google.com/github/MarkStephens060482/forecasting_BEV_adoption_LGA/blob/main/Uptake_of_Electric_Vehicles_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uptake of Electric Vehicles at LGA level - Exploratory Data Analysis
### Enhancing Sustainable Transportation Planning through Explainable Electric Vehicle Adoption Forecasting in Ausrralian Local Governments.
1. Define Outcome Variable, BEV share per 1000 people.
1. Examine spatial representation of outcome variable, represent BEV share at LGA level on map of Australia and examine time variation.

In [ ]:
!pip install geopandas plotly

Import modules and set path

In [ ]:
import geopandas as gpd
import numpy as np, pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import drive
import os
from pathlib import Path
from IPython.core.display import HTML
from matplotlib import pyplot as plt

drive.mount('/content/drive')

#%load_ext cudf.pandas
# Set the working directory to a specific path
cwd = os.getcwd()
PATH = "/content/drive/MyDrive/Masters Data Science work/Masters of Data Science Study/Masters Program/Capstone Project"
os.chdir(PATH)
# Define the new directory path
new_dir = Path(cwd +"/EDA")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Clean data set and prepare for analysis

In [ ]:
# Change the current working directory to the new directory
dfs=[]
for file in list(new_dir.glob("*.csv")):
  with open(file,encoding="latin1") as dataFile:
        df = pd.read_csv(dataFile,index_col=False)
  dfs.append(df)

# define EV
EV_df = dfs[0]
EV_df['EV_prop'] = 1000*EV_df['Electric']/EV_df['Total']
EV_df['Year'] = EV_df['Year'].astype(int)
EV_df = EV_df[~EV_df['LGA_code'].isin([19399,49399])]

#define charging
charging_df = dfs[1]

# Read the shapefile
geodf = gpd.read_file(list(new_dir.glob("*.shp"))[0])
# shape file is a different CRS,  change to lon/lat GPS co-ordinates
geodf = geodf.to_crs(epsg=4326)
# simplify geometry to 250m accuracy
geodf["geometry"] = (
    geodf.to_crs(geodf.estimate_utm_crs()).simplify(250).to_crs(geodf.crs)
)

#Clean geodf
geodf = geodf.rename(columns={"LGA_CODE22": "LGA_code",
                             "LGA_NAME22": "LGA_name"})
EV_df = EV_df.astype({"LGA_code": str}) # convert LGA_code to int
geodf = geodf[(geodf["LGA_code"].isin(EV_df["LGA_code"]))]

# set LGA_code as index
geodf = geodf.set_index("LGA_code")
EV_df = EV_df.set_index("LGA_code")

### Data Description and distributions
1. Examine scatter plot matrix of variables from ABS Population and Housing Census dataset with the Outcome Variable


In [ ]:
print(EV_df.info())
EV_df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 7020 entries, 10050 to 89399
Data columns (total 76 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 7020 non-null   int64  
 1   Fast_DC_chargers     7020 non-null   int64  
 2   BD_3                 4572 non-null   float64
 3   CABEE_10             3696 non-null   float64
 4   CABEE_15             3679 non-null   float64
 5   CABEE_17             4677 non-null   float64
 6   CABEE_18             4535 non-null   float64
 7   CABEE_19             4739 non-null   float64
 8   CABEE_20             4190 non-null   float64
 9   CABEE_21             4750 non-null   float64
 10  CABEE_22             4188 non-null   float64
 11  CABEE_23             3993 non-null   float64
 12  CABEE_24             4479 non-null   float64
 13  CABEE_25             4436 non-null   float64
 14  CABEE_26             3888 non-null   float64
 15  CABEE_27             4541 non-null   f

,Year,Fast_DC_chargers,BD_3,CABEE_10,CABEE_15,CABEE_17,CABEE_18,CABEE_19,CABEE_20,CABEE_21,...,SCHOOL_4,SCHOOL_5,SOLAR_2,STRESS_14,WORK_TRAV_5,Longitude,Latitude,Electric,Total,EV_prop
count,7020.000000,7020.000000,4572.000000,3696.000000,3679.000000,4677.000000,4535.000000,4739.000000,4190.000000,4750.000000,...,1582.000000,1619.000000,4686.000000,1597.000000,1620.000000,7020.000000,7020.000000,7020.000000,7020.000000,7003.000000
mean,2017.000000,0.283048,2.058185,661.737202,551.306836,185.185996,190.411383,326.860000,65.267774,728.520327,...,1.591488,10.730006,4409.841795,86.852064,12338.807711,-31.438626,138.035306,25.022222,25787.279630,0.495656
std,3.741924,1.036352,0.463462,1538.568722,1235.972811,392.897449,441.006342,370.977583,141.817899,1446.084136,...,0.904558,6.763951,9751.158760,11.720367,24232.403588,6.793610,13.127745,140.921093,50738.672047,1.362817
min,2011.000000,0.000000,0.331087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.199999,0.400000,3.000000,14.430699,16.000000,-43.251253,96.857741,0.000000,0.000000,0.000000
25%,2014.000000,0.000000,1.800000,32.000000,30.000000,15.000000,8.000000,81.000000,4.000000,34.000000,...,1.000000,6.700000,342.250000,77.100000,672.500000,-35.162608,126.997677,0.000000,1295.000000,0.000000
50%,2017.000000,0.000000,2.020000,137.000000,128.000000,54.000000,36.000000,189.000000,17.000000,174.500000,...,1.300000,8.500000,1388.000000,92.100000,3601.500000,-33.187324,143.687125,0.000000,7178.000000,0.000000
75%,2020.000000,0.000000,2.290000,604.000000,528.500000,203.000000,181.000000,465.500000,69.000000,763.000000,...,1.999989,11.900000,4525.000000,95.000000,13054.500000,-29.437660,147.873323,8.000000,27327.000000,0.393756
max,2023.000000,34.000000,5.300000,24246.000000,20225.000000,5972.000000,7026.000000,3622.000000,2133.000000,17746.000000,...,5.200000,37.100000,181279.000000,116.700000,344563.000000,-9.859244,153.486170,6680.000000,795288.000000,21.038436


In [ ]:
for i in range(0, len(EV_df.columns), 10):  # Step through the list in increments of 10
    print(', '.join(map(str, EV_df.columns[i:i+10])))  # Print 10 items per row
    # Create the scatter matrix
    # Number of variables per scatter matrix

columns = list(EV_df.columns[:68]) + [EV_df.columns[-1]] + [EV_df.columns[-4]]

# Select these columns using loc
selected_columns_df = EV_df.loc[:, columns]
Year = 2021
selected_columns_df = selected_columns_df[selected_columns_df.Year == Year]

dimensions = selected_columns_df[selected_columns_df.columns[-10:-1]].columns
# Create scatter matrix for these 10 variables
scatter_matrix_fig = px.scatter_matrix(selected_columns_df, dimensions=dimensions, color = "Electoral Rating")

# Rotate dimension labels
for annotation in scatter_matrix_fig.layout.annotations:
    annotation.update(textangle=45)  # Adjust the angle as needed

# Update layout to set the title and adjust size
scatter_matrix_fig.update_layout(
    title_text=f"Scatter Matrix of 10 Variables with Outcome Variable for Year {Year}",
    height=850,  # Set the height of the figure
    width=1000    # Set the width of the figure
)
# Show the plot
scatter_matrix_fig.show()

Year, Fast_DC_chargers, BD_3, CABEE_10, CABEE_15, CABEE_17, CABEE_18, CABEE_19, CABEE_20, CABEE_21
CABEE_22, CABEE_23, CABEE_24, CABEE_25, CABEE_26, CABEE_27, CABEE_28, CABEE_30, CABEE_31, CABEE_32
CABEE_33, CABEE_34, CABEE_36, CABEE_37, CABEE_5, CENSUS_13, CHILD_2, ERP_21, ERP_P_10, ERP_P_11
ERP_P_12, ERP_P_13, ERP_P_14, ERP_P_15, ERP_P_16, ERP_P_17, ERP_P_18, ERP_P_19, ERP_P_2, ERP_P_20
ERP_P_3, ERP_P_4, ERP_P_5, ERP_P_6, ERP_P_7, ERP_P_8, ERP_P_9, HHTYPE_5, HHTYPE_6, HIGH_2
HOUSES_2, HOUSES_3, INCOME_10, INCOME_11, INCOME_12, INCOME_13, INCOME_2, INCOME_3, INCOME_9, PENSION_3
PENSION_4, PENSION_5, SCHOOL_3, SCHOOL_4, SCHOOL_5, SOLAR_2, STRESS_14, WORK_TRAV_5, Longitude, Latitude
Official Name State, LGA_name, Electoral Rating, Electric, Total, EV_prop


### Chloropleth of BEV share per 1000 people at the LGA level

In [ ]:
# Plot
map_fig = px.choropleth_mapbox(
    EV_df,
    geojson = geodf.geometry,
    locations = EV_df.index,
    color="EV_prop",
    color_continuous_scale="Reds",
    range_color=(0, EV_df.EV_prop.max()),
    animation_frame="Year",
    labels={"EV_prop": "Passenger BEV share per 1000"},
    hover_name="LGA_name",
    opacity=0.6,
    center=dict(lat=-37.632797, lon=144.21561), # Moorabool Shire
    mapbox_style="open-street-map",
    #mapbox_style="carto-positron",
    zoom=9,
)
animate_duration = 1000  # Duration of the animation in milliseconds

map_fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': animate_duration, 'redraw': True}, 'fromcurrent': True}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }],
    sliders=[{
        'currentvalue': {
            'prefix': 'Year: ',
            'font': {'color': 'blue', 'size': 20}
        },
        'steps': [{'args': [[f.name], {'frame': {'duration': animate_duration, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                   'label': f.name,
                   'method': 'animate'} for f in map_fig.frames],
        'pad': {'b': 20},
        'len': 0.9
    }],
    title={
        'text': "Uptake of Battery Electric Vehicles  in Local Government Areas",
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Arial",
        size=20,
        color='blue'
    ),
    width=1700,
    height=875,
    autosize=True,
    margin={"r": 10, "t": 30, "l": 10, "b": 30},
    legend=dict(
        x=0.01,  # Position legend to the left
        y=0.95,  # Position legend to the top
        bgcolor='rgba(255, 255, 255, 0.5)',  # Background color with transparency
        bordercolor='Black',
        borderwidth=2
    ),
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial")
    )

# Save the plot as an HTML file
output_file = 'choropleth_mapbox.html'
map_fig.write_html(output_file)


In [ ]:
#plot a density plot of EV_prop
# Create the density plot with animation
fig = px.histogram(
    EV_df,
    x='EV_prop',
    animation_frame='Year',
    title='Histogram of BEV share per 1000 over years.',
    labels={'EV_prop': 'BEV share per 1000 people'},
    range_x=[EV_df['EV_prop'].min(), EV_df['EV_prop'].max()],
    range_y=[0, EV_df['EV_prop'].value_counts().max()]
)


# Update the layout for better visual appeal
fig.update_layout(
    xaxis_title='BEV share per 1000 people',
    yaxis_title='Density',
    margin={"r":0,"t":40,"l":0,"b":100},
    width=1600,
    height=875,
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }],
    sliders=[{
        'currentvalue': {'prefix': 'Year: ', 'font': {'size': 20}},
        'steps': [{'args': [[f.name], {'frame': {'duration': 500, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                   'label': f.name,
                   'method': 'animate'} for f in fig.frames]
    }]
)


In [ ]:
#plot a parallel boxplot of EV_prop grouped by Electoral Rating
# Define the desired category order
category_order = ['Inner Metropolitan', 'Outer Metropolitan','Provincial','Rural']

# Create the animated parallel boxplot
fig = px.box(EV_df,
             x='Electoral Rating',
             y='EV_prop',
             color='Electoral Rating',
             animation_frame='Year',
             range_y=[0, EV_df['EV_prop'].max()],
             hover_data={'LGA_name': True, 'Official Name State': True},
             category_orders={'Electoral Rating': category_order})

# Update layout to improve aesthetics
fig.update_layout(
    xaxis_title="Electoral Ratings",
    yaxis_title="BEV share per 1000 people",
    boxmode="group",
    title={
        'text': "Distributions of BEV share per 1000 in LGAs across electoral ratings from 2013 to 2023",
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Arial",
        size=20,
        color='blue'
    ),
    width=1400,
    height=875,
    autosize=True,
    margin={"r": 10, "t": 30, "l": 10, "b": 30},
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 1000, 'redraw': True}, 'fromcurrent': True}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }],
    sliders=[{
        'currentvalue': {'prefix': 'Year: ', 'font': {'size': 20}},
        'steps': [{'args': [[f.name], {'frame': {'duration': 1000, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                   'label': f.name,
                   'method': 'animate'} for f in fig.frames]
    }]
)
# Save the plot as an HTML file
output_file = 'EV_prop distribution.html'
fig.write_html(output_file)

In [ ]:
mean_chargers_per_LGA = EV_df.groupby(by = ['Official Name State','Year'])[['Fast_DC_chargers','EV_prop']].mean().rename(columns={'Fast_DC_chargers': 'Fast_DC_chargers_mean','EV_prop': 'EV_prop_mean'})
lga_counts = EV_df.groupby(by = ['Official Name State','Year'])[['Fast_DC_chargers','EV_prop']].count()
err_chargers_per_LGA =(EV_df.groupby(by = ['Official Name State','Year'])[['Fast_DC_chargers','EV_prop']].std()/lga_counts**(1/2)).rename(columns={'Fast_DC_chargers': 'Fast_DC_chargers_err','EV_prop': 'EV_prop_err'})
# join the grouped dataframes
EVprop_chargers = pd.merge(mean_chargers_per_LGA,err_chargers_per_LGA,left_index=True,right_index=True).reset_index()
# filter
EVprop_chargers = EVprop_chargers[~(EVprop_chargers['Official Name State'].isin(['Australian Capital Territory','Other Territories'])) & (EVprop_chargers['Year'] >= 2013 )]

# produce a Plotly scatter plot between Fast_DC_chargers and EV_prop, grouped by Electoral Rating from the EV_df dataframe, with animation frame of Years.
# Create the scatter plot
fig = px.scatter(EVprop_chargers,
                 x='Fast_DC_chargers_mean',
                 y='EV_prop_mean',
                 animation_frame ='Official Name State',
                 color = 'Official Name State',
                 hover_name = 'Official Name State',
                 error_x='Fast_DC_chargers_err', error_y='EV_prop_err',
                 trendline='ols',
                 hover_data={'Year': True},
                 # title="Relationship between of amount of Fast DC chargers and BEV share per 1000 people over years."
                 )

# Set the x-axis limits
fig.update_xaxes(range=[EVprop_chargers['Fast_DC_chargers_mean'].min(), EVprop_chargers['Fast_DC_chargers_mean'].max()])
# Set the y-axis limits
fig.update_yaxes(range=[EVprop_chargers['EV_prop_mean'].min(), EVprop_chargers['EV_prop_mean'].max()])

# Update layout to improve aesthetics
fig.update_layout(
    xaxis_title="mean number of Fast DC charging stations across LGAs",
    yaxis_title="mean BEV share per 1000 people across LGAs",
    boxmode="group",
    title={
        'text': "Relationship between average number of Fast DC chargers in LGAs and average BEV share per 1000 people in LGAs over years for each state.",
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        family="Arial",
        size=20,
        color='blue'
    ),
    showlegend=False,  # Hide the legend
    width=1400,
    height=800,
    autosize=True,
    margin={"r": 10, "t": 30, "l": 10, "b": 30},
    paper_bgcolor='white',
    #plot_bgcolor='white',
    )
fig.show()

Fast_DC_chargers_err  EV_prop_err
Official Name State          Year                                   
Australian Capital Territory 2011                   NaN          NaN
                             2012                   NaN          NaN
                             2013                   NaN          NaN
                             2014                   NaN          NaN
                             2015                   NaN          NaN
...                                                 ...          ...
Western Australia            2019              0.022624     0.120785
                             2020              0.036670     0.062966
                             2021              0.044361     0.114170
                             2022              0.053477     0.178470
                             2023              0.075893     0.304230

[117 rows x 2 columns]